In [188]:
import pydistinct
import psycopg2
import pandas as pd

In [189]:
conn = psycopg2.connect(f"dbname=nba_study user=postgres port=5433")
conn.autocommit = True
cur = conn.cursor()

In [190]:
# sample_mv_01 = "CREATE MATERIALIZED VIEW sample_01 AS SELECT * FROM jg_196 ORDER BY RANDOM() limit 260";
# sample_mv_03 = "CREATE MATERIALIZED VIEW sample_03 AS SELECT * FROM jg_196 ORDER BY RANDOM() limit 520";
# sample_mv_05 = "CREATE MATERIALIZED VIEW sample_05 AS SELECT * FROM jg_196 ORDER BY RANDOM() limit 1300";
# sample_mv_07 = "CREATE MATERIALIZED VIEW sample_07 AS SELECT * FROM jg_196 ORDER BY RANDOM() limit 1800";

In [191]:
# cur.execute(sample_mv_01)
# cur.execute(sample_mv_03)
# cur.execute(sample_mv_05)
# cur.execute(sample_mv_07)

In [192]:
sample_q_01 = "SELECT * FROM sample_01";
sample_q_03 = "SELECT * FROM sample_03";
sample_q_05 = "SELECT * FROM sample_05";
sample_q_07 = "SELECT * FROM sample_07";


In [193]:
s_df_01 = pd.read_sql(sample_q_01, conn)
s_df_03 = pd.read_sql(sample_q_03, conn)
s_df_05 = pd.read_sql(sample_q_05, conn)


In [194]:
s_df_01.head()

,win,season_name,a_1,a_2,a_3,a_4,a_5,a_6,a_7,a_8,...,a_24,a_25,a_26,a_27,a_28,a_29,a_30,a_31,a_32,a_33
0,73,2015-16,2,GSW,2016-03-12,123,116,102,101,2,...,0.33,0.54,21.88,82,0,3,3,0,Mirza Teletovic,82
1,73,2015-16,2,GSW,2016-01-08,108,128,106,106,16,...,0.33,0.32,18.97,315,3,0,0,0,Harrison Barnes,315
2,73,2015-16,2,GSW,2015-12-05,109,112,92,92,8,...,0.00,0.40,7.35,342,1,10,6,4,Patrick Patterson,342
3,73,2015-16,2,GSW,2015-11-07,94,103,99,100,11,...,0.00,0.50,15.63,467,7,9,5,4,Draymond Green,467
4,47,2012-13,2,GSW,2012-10-31,85,87,92,93,13,...,0.50,0.36,26.53,76,1,4,3,1,Michael Beasley,76


In [195]:
d_pattern = "a_32='Stephen Curry'"

q_recall_full = f"""
WITH tp AS(
SELECT COUNT(DISTINCT pnumber) as tp
FROM jg_196 
WHERE {d_pattern} and is_user = 'yes'
)
SELECT t.tp::numeric/COUNT(DISTINCT pnumber) as recall 
FROM jg_196, tp t
WHERE is_user = 'yes'
GROUP BY t.tp
"""
cur.execute(q_recall_full)
true_recall = float(cur.fetchone()[0])
print(f"true recall : {true_recall}")

true recall : 0.9726027397260274


In [196]:

q_recall_01 = f"""
WITH tp AS(
SELECT COUNT(DISTINCT pnumber) as tp
FROM sample_01 
WHERE {d_pattern} and is_user = 'yes'
)
SELECT t.tp::numeric/COUNT(DISTINCT pnumber) as recall 
FROM sample_01, tp t
WHERE is_user = 'yes'
GROUP BY t.tp
"""
cur.execute(q_recall_01)
sampe_recall_result_01 = float(cur.fetchone()[0])
print(f"sample 0.1 recall result : {sampe_recall_result_01}")

sample 0.1 recall result : 0.109375


In [197]:
q_recall_03 = f"""
WITH tp AS(
SELECT COUNT(DISTINCT pnumber) as tp
FROM sample_03 
WHERE {d_pattern} and is_user = 'yes'
)
SELECT t.tp::numeric/COUNT(DISTINCT pnumber) as recall 
FROM sample_03, tp t
WHERE is_user = 'yes'
GROUP BY t.tp
"""
cur.execute(q_recall_03)
sampe_recall_result_03 = float(cur.fetchone()[0])
print(f"sample 0.3 recall result : {sampe_recall_result_03}")

sample 0.3 recall result : 0.2112676056338028


In [198]:
q_recall_05 = f"""
WITH tp AS(
SELECT COUNT(DISTINCT pnumber) as tp
FROM sample_05 
WHERE {d_pattern} and is_user = 'yes'
)
SELECT t.tp::numeric/COUNT(DISTINCT pnumber) as recall 
FROM sample_05, tp t
WHERE is_user = 'yes'
GROUP BY t.tp
"""
cur.execute(q_recall_05)
sampe_recall_result_05 = float(cur.fetchone()[0])
print(f"sample 0.5 recall result : {sampe_recall_result_05}")

sample 0.5 recall result : 0.4794520547945205


In [199]:
q_recall_07 = f"""
WITH tp AS(
SELECT COUNT(DISTINCT pnumber) as tp
FROM sample_07 
WHERE {d_pattern} and is_user = 'yes'
)
SELECT t.tp::numeric/COUNT(DISTINCT pnumber) as recall 
FROM sample_07, tp t
WHERE is_user = 'yes'
GROUP BY t.tp
"""
cur.execute(q_recall_07)
sampe_recall_result_07 = float(cur.fetchone()[0])
print(f"sample 0.7 recall result : {sampe_recall_result_07}")

sample 0.7 recall result : 0.6164383561643836


In [200]:

a_32_q = "SELECT a_32 from sample_01 where is_user='yes'"

x = pd.read_sql(a_32_q, conn)

r = x['a_32'].value_counts().to_dict()


horvitz_thompson = pydistinct.stats_estimators.horvitz_thompson_estimator(attributes=r, n_pop=2600)
smoothed_jackknife = pydistinct.stats_estimators.smoothed_jackknife_estimator(attributes=r, n_pop=2600)

u_horvitz_thompson = pydistinct.stats_estimators.horvitz_thompson_estimator(attributes=r)
u_smoothed_jackknife = pydistinct.stats_estimators.smoothed_jackknife_estimator(attributes=r)

print(f"sample 10% known pop: horvitz_thompson = {horvitz_thompson}")
print(f"sample 10% known pop: smoothed_jackknife = {smoothed_jackknife}")

print(f"sample 10% unknown pop: horvitz_thompson = {u_horvitz_thompson}")
print(f"sample 10% unknown pop: smoothed_jackknife = {u_smoothed_jackknife}")

sample 10% known pop: horvitz_thompson = 118.49462944511886
sample 10% known pop: smoothed_jackknife = 134.00484267144986
sample 10% unknown pop: horvitz_thompson = 120.3003577601435
sample 10% unknown pop: smoothed_jackknife = 138.86812246862766


In [201]:
a_32_q = "SELECT a_32 from sample_03 where is_user='yes'"

x = pd.read_sql(a_32_q, conn)

r = x['a_32'].value_counts().to_dict()



horvitz_thompson = pydistinct.stats_estimators.horvitz_thompson_estimator(attributes=r, n_pop=2600)
smoothed_jackknife = pydistinct.stats_estimators.smoothed_jackknife_estimator(attributes=r, n_pop=2600)


u_horvitz_thompson = pydistinct.stats_estimators.horvitz_thompson_estimator(attributes=r)
u_smoothed_jackknife = pydistinct.stats_estimators.smoothed_jackknife_estimator(attributes=r)

print(f"sample 30% known pop: horvitz_thompson = {horvitz_thompson}")
print(f"sample 30% known pop: smoothed_jackknife = {smoothed_jackknife}")

print(f"sample 30% unknown pop: horvitz_thompson = {u_horvitz_thompson}")
print(f"sample 30% unknown pop: smoothed_jackknife = {u_smoothed_jackknife}")



sample 30% known pop: horvitz_thompson = 199.70539615580518
sample 30% known pop: smoothed_jackknife = 216.06410150310356
sample 30% unknown pop: horvitz_thompson = 211.0812455763181
sample 30% unknown pop: smoothed_jackknife = 226.24351034693626


In [202]:
a_32_q = "SELECT a_32 from sample_05 where is_user = 'yes'"

x = pd.read_sql(a_32_q, conn)

r = x['a_32'].value_counts().to_dict()


horvitz_thompson_05 = pydistinct.stats_estimators.horvitz_thompson_estimator(attributes=r, n_pop=2600)
smoothed_jackknife_05 = pydistinct.stats_estimators.smoothed_jackknife_estimator(attributes=r, n_pop=2600)


u_horvitz_thompson_05 = pydistinct.stats_estimators.horvitz_thompson_estimator(attributes=r)
u_smoothed_jackknife_05 = pydistinct.stats_estimators.smoothed_jackknife_estimator(attributes=r)

print(f"sample 50% known pop: horvitz_thompson = {horvitz_thompson_05}")
print(f"sample 50% known pop: smoothed_jackknife = {smoothed_jackknife_05}")

print(f"sample 50% unknown pop: horvitz_thompson = {u_horvitz_thompson_05}")
print(f"sample 50% unknown pop: smoothed_jackknife = {u_smoothed_jackknife_05}")


sample 50% known pop: horvitz_thompson = 352.6083861419974
sample 50% known pop: smoothed_jackknife = 377.14922758403975
sample 50% unknown pop: horvitz_thompson = 412.0673354084649
sample 50% unknown pop: smoothed_jackknife = 376.4127623377631


In [203]:
a_32_q = "SELECT a_32 from sample_07 where is_user = 'yes'"

x = pd.read_sql(a_32_q, conn)

r = x['a_32'].value_counts().to_dict()


horvitz_thompson_07 = pydistinct.stats_estimators.horvitz_thompson_estimator(attributes=r, n_pop=2600)
smoothed_jackknife_07 = pydistinct.stats_estimators.smoothed_jackknife_estimator(attributes=r, n_pop=2600)


u_horvitz_thompson_07 = pydistinct.stats_estimators.horvitz_thompson_estimator(attributes=r)
u_smoothed_jackknife_07 = pydistinct.stats_estimators.smoothed_jackknife_estimator(attributes=r)

print(f"sample 70% known pop: horvitz_thompson = {horvitz_thompson_07}")
print(f"sample 70% known pop: smoothed_jackknife = {smoothed_jackknife_07}")

print(f"sample 70% unknown pop: horvitz_thompson = {u_horvitz_thompson_07}")
print(f"sample 70% unknown pop: smoothed_jackknife = {u_smoothed_jackknife_07}")

sample 70% known pop: horvitz_thompson = 391.51404275776633
sample 70% known pop: smoothed_jackknife = 414.41588623760254
sample 70% unknown pop: horvitz_thompson = 463.09748808068264
sample 70% unknown pop: smoothed_jackknife = 405.89777996525567


In [204]:
a_32_distinct_q = "SELECT count(distinct a_32) from jg_196 where is_user = 'yes'"
yes_cnt_q = "SELECT count(*) from jg_196 where is_user= 'yes'"

cur.execute(a_32_distinct_q)
true_distinct = float(cur.fetchone()[0])

cur.execute(yes_cnt_q)
yes_cnt = float(cur.fetchone()[0])

print(f"true distinct a_32: {true_distinct}")

print(f"yes cnt: {yes_cnt}")

true distinct a_32: 390.0
yes cnt: 1612.0
